<pre>
<h1>
Binary Search Tree, BST
</h1>
[root, size, height] --------> Node()

보통 이진트리에서 값을 찾으려면 순회하며 모든 노드를 돌아다녀야 했다. 뽈뽈뽈
이때 세가지 방법이 있었다. preorder, inorder, postorder

하지만, 애초에 규칙을 정해두고 이진트리를 만든다면? 찾기도 쉬울것이다.
따라서, 탐색 연산의 효율성을 위해 일정 규칙에 맞게 값을 삽입할 수 있다.

<h1>
1. Node Class
</h1>
- Attributes
    `parent`, `key`, (height), `left`, `right`

- Methods
    Dunder Methods
    `__init__` 
    `__iter__`
    `__str__`


<h1>
2. BST Class
</h1>
- Rules 
    1) None값은 하나의 텅 빈 BST로 취급한다.
    2) BST의 노드를 v라고 했을때, v는 왼쪽 자식노드(left)보다 커야하고, 오른쪽 자식노드(right)보다는 작아야한다. 
       left < v < right

- Attributes
    `root`, `size`, `height`

- Methods
    Dunder Methods
    `__init__`        => root, size, (height) 
    `__len__`
    `__iter__`        => Node Class에 정의한 이터레이터 호출
    `__str__`         


    Instance Methods
    `preorder(v)`
    `inorder(v)`
    `postorder(v)`
    
    `findLocation(key)`   => hashTable을 구현할 때, 어떤 key값이 들어갈 자리, 혹은 이미 들어있는 자리를 찾았던 것처럼(findSlot)
                        => (들어간 자리) 규칙에 따라 내려갔을 떄 일치하는 key값이 존재하면 해당 노드를 반환하고, 
                        => (들어갈 자리) 비어있으면 삽입될 곳의 부모를 리턴
    `search(key)`         => v와 v의 자손노드 중에서 주어진 key 값과 일치하는 노드를 찾고(findLocation) 없다면 None리턴
    `insert(key)`         => key가 들어갈 자리가 정해져있으므로, findLocation을 호출
                        => key값이 찾은 노드의 키와 일치하지 않고, None이 아니라면 내가 들어갈 부모노드
                            => 비교해서 왼쪽이나 오른쪽에 삽입
    `updateNodeHeight(v)` => 노드 v의 현재 높이 수정
    `updateHeight(v)`     => v부터 root까지 모든 노드의 높이 수정 => updateNodeHeight(v) 호출
    `deleteByMerging(x)`  => 삭제 후 남은 자식들을 기존의 자리에 병합 
                        => 상황 : x를 삭제하려고 한다. (삭제할 노드를 x, x의 부모를 pt, x의 left를 a, right를 b라고 하자)
                            case1. a노드의 존재 유무 (x의 빈자리 쟁탈전)
                                => 존재한다 
                                    => a를 기존의 x자리에 앉힌다.
                                    => b는? a의 오른쪽 자식으로 보낸다
                                => 존재하지 않는다
                                    => b를 기존의 x자리에 앉힌다.  
                            case2. 지울 노드가 root인지 여부
                                => root다
                                    => 난 너같은 자식 둔 적 없다 시전
                                => root가 아니다
                                    => 상호 완만한 해결을 권장합니다. (난 너무 가난해)

    `deleteByCopying(x)`

    """
    ToDo
    트리 자체를 보는 게 쉽지 않으니, 보기 편하게 만들기 ok
    deleteByCoping 버전은 아직 구현 전. 
    삭제할 노드가 root인 경우 등 더 테스트 케이스 추가
    insertAVL의 test case 추가

    READ ME 추가 어떤식으로 만들어야할까

    """
</pre>

In [103]:

class Node:
    def __init__(self, key):
        self.key = key
        self.height = 0
        self.parent = None
        self.left = None
        self.right = None

    # inorder 방식으로 순회 => BST에서 노드의 요소들이 오름차순으로 정렬됨
    def __iter__(self):
        if self != None:
            if self.left != None:
                for leftSubTree in self.left:
                    yield leftSubTree
            yield self
            if self.right != None: 
                for rightSubTree in self.right:
                    yield rightSubTree

    def __str__(self):
        return str(self.key)

class BST:
    def __init__(self):
        self.root = None
        self.size = 0
        self.height = self.updateTreeHeight()
        
    def updateTreeHeight(self):
        self.height = max(i.height for i in self.root) if self.root != None else 0

    def __len__(self):
        return self.size
    
    def __str__(self):
        if self.root:
            return ("\n").join("h"+str(i.height)+" : "+str(i) for i in self.root)
        else:
            return "empty"

    def preorder(self, v):
        if v != None:
            print(v, end=" ")
            if v.left != None:
                self.preorder(v.left)
            if v.right != None:
                self.preorder(v.right)

    def inorder(self, v):
        if v != None:
            if v.left != None:
                self.inorder(v.left)
            print(v, end=" ")
            if v.right != None:
                self.inorder(v.right)

    def postorder(self, v):
        if v != None:
            if v.left != None:
                self.postorder(v.left)
            if v.right != None:
                self.postorder(v.right)
            print(v, end=" ")

    def findLocation(self, key):
        # 들어있는 자리를 찾거나, 내가 들어갈 자리를 확인 후, 부모를 호출(그래야 부모의 자식으로 연결할 수 있으니까)
        # 근데 애초에 빈 트리면 부모도 내자리도 아무것도 없으니 None
        if self.size == 0: return None
        # running tech을 사용하여, 어떤 노드 v와 v의 부모노드인 p를 동시에 찾는다.
        p = None 
        v = self.root
        # v가 None이 되면 
        while v:
            # key값이 일치하는 v를 찾았다면 반환
            if v.key == key:
                return v
            # 찾고자 하는 key가 v의 key값보다 크다면 오른쪽 탐색
            elif v.key < key:
                p = v
                v = v.right
            # 찾고자 하는 key가 v보다 작으면 왼쪽 탐색
            else:
                p = v
                v = v.left
        # while문을 빠져나왔다는 것은, 찾고자 하는 key를 가진 v가 존재하지 않고, 해당 노드가 들어갈 자리만 찾았다는 말
        # 따라서 v가 들어갈 수 있는 자리의 부모노드 반환
        return p
    
    # 사실 findLocation의 일정부분까지만 사용하면 search를 구현할 수 있지만, 코드의 재사용성을 위해 
    # 왜냐면 해당 노드를 찾는다고 해도, key값이 들어갈 자리의 부모를 찾기 위해 똑같은 걸 반복해야하니까.
    def search(self, key):
        #findLocation을 호출하여, 해당 key가 들어가있거나, 들어갈 자리의 부모노드를 호출(만약 아무것도 없으면 None)
        p = self.findLocation(key)
        if p and p.key == key:
            return p
        else: 
            return None
    # 노드 v의 높이를 수정
    def update_node_height(self, v):
        # v가 존재한다면,
        if v:
            # 해당 노드의 자식이 존재하는지 검사 후, 자식 중에 더 깊이 뻗어있는 자식을 선택해서 현재 높이를 업데이트
            l = v.left.height if v.left else -1
            r = v.right.height if v.right else -1
            # 최대 높이를 가진 자식보단 본인이 한칸 더 크니까
            v.height = max(l, r) + 1

    # v부터 root가지 올라가면서 모든 노드 정보 업데이트
    # 노드를 삽입할 때마다 호출한다면, 언제나 리프노드부터 루트까지 존재하는 높이들을 업데이트 가능
    def update_height(self, v):
        # v가 루트노드에 도달할때까지 
        while v != None:
            # 높이를 갱신하고, 부모노드로 현재노드 이동
            self.update_node_height(v)
            v = v.parent
        self.updateTreeHeight()
        
    def insert(self, key):
        p = self.findLocation(key)
        # 삽입이 가능한 경우 : 빈 노드에 첫 삽입이거나, 부모 노드가 반환된 경우
        if p == None or p.key != key:
            # print("검색 성공")
            newNode = Node(key)

            # 첫 삽입인 경우
            if p == None:
                # print("첫번째 삽입")
                self.root = newNode
            # 내가 들어갈 부모 노드를 찾은 경우
            else:
                # print("하나 이상의 노드가 존재하는 트리에 삽입 시도")
                newNode.parent = p
                # 부모보다 작은 값은 왼쪽에 삽입
                if p.key > key:
                    # print("왼쪽 자식에 연결")
                    p.left = newNode
                # 부모보다 큰 경우 오른쪽
                else:
                    # print("오른쪽 자식에 연결")
                    p.right = newNode
            self.size += 1
            self.update_height(newNode)
        # 삽입하고자 하는 key값이 이미 존재하는 경우
            return newNode   
        else:
            # print("이미 중복된 값이 존재")
            # 중복 허용 안한다면 none
            return None


    def deleteByMerging(self, key):
        x = self.search(key)
        if x == None: return None
        else: pass
        # 상황 : x를 삭제하려고 한다. (삭제할 노드를 x, x의 부모를 pt, x의 left를 a, right를 b라고 하자)
        xL, xR, pt = x.left, x.right, x.parent
        unBalancedPoint = None
        # case1. a노드의 존재 유무 (x의 빈자리 쟁탈전)
        if xL == None:
        # x자리에 올 노드를 c라고 부르자. (일종의 c를 두고 xL과 xR가 대결하는 구도) => xL의 존재여부만 가림, xR는 2차전에서 deletePoint의 존재 여부로 확인 
            #xL이 없으니까, xR가 deletePoint자리 차지(원래 xR는 xL의 아래로 가야함)
            deletePoint = xR            #deletePoint는 c로도 표현
            unBalancedPoint = pt
        else:
            # 왕좌를 차지한 xL
            deletePoint = xL 
            # xL의 가장 오른쪽 리프 노드를 찾기 위해 xL의 암행어사 rightLeaf 생성
            rightLeaf = xL
            # 그런 뒤에 xL의 가장 오른쪽 리프노드 rightLeaf을 찾고
            while rightLeaf.right != None :
                rightLeaf = rightLeaf.right
            # xR는 처절하게 rightLeaf을 부모로 인정함
            if xR != None:
                xR.parent = rightLeaf
            # xR가 존재하거나 말거나, 암행어사 rightLeaf은 (xR를 자식으로 삼음)
            rightLeaf.right = xR
            unBalancedPoint = rightLeaf
        #왕좌에 앉기 게임의 1차전이 끝났고 이제 선대에게 인정받아야함
        
        # case2. 지울 노드가 root인지 여부
        # 루트노드를 지운 게 아닌 경우 (루트 갱신 불필요)
        if self.root != x:
            # 근데 만약 1차전에서 xL도 없고 xR도 없었다면 deletePoint는 None일 수도 있으니까 일단 살아있는지 확인해야함
            if deletePoint:
                # 선대를 자신의 부모로 인정함
                deletePoint.parent = pt
                # 부모도 새로운 왕을 인정함
                if  deletePoint.key < pt.key:
                    pt.left = deletePoint
                else:
                    pt.right = deletePoint
            # 근데 1차전에서 다 죽었다면 기존 x자리를 빈자리로 만들어야함 deletePioint = None
            else:
                if pt.left == x:
                    pt.left = deletePoint
                else:
                    pt.right = deletePoint
        # 만약 지운 값이 root였다면? (루트 갱신 필요)
        else:
            # 일단 루트 자리에 새로운 c가 옴
            self.root = deletePoint
            # c가 있다면 이전에 섬기던 왕 x를 잊고, 자신이 직접 왕이 되어야함(아무도 의지하지마..!
            if deletePoint != None: 
                deletePoint.parent = None
            
        self.size -= 1
        # a와 b가 모두 존재하는 경우, a의 암행어사로서, 가장 오른쪽 리프노드였던 b는 왼쪽 자식은 있었을 수도 있고, 오른쪽 자식은 없었다. (자신이 오른쪽 끝이니까)
        # 그렇다면 m의 오른자식으로 b를 붙였을 때, m의 높이 정보는 m.left와(있었다면) m.right(b를 포함한 자식들) 과 비교를 해서 쭉 root까지만 업데이트하면 된다.
        print("Unbalanced Point : ", unBalancedPoint)
        self.update_height(unBalancedPoint)
        return unBalancedPoint 
    
    # fixed
    def print_binary_tree_structure(self, node, prefix="", is_left=True, is_root=True):
        if node is not None:
            if is_root:
                print(f"──── {node}(h={node.height})")
            else:
                print(prefix + (" ├── " if is_left else " └── ") + f"{node}(h={node.height})")
                
            if node.left or node.right :
                if node.left:  # 왼쪽 자식이 있는 경우
                    self.print_binary_tree_structure(node.left, prefix + (" │    " if not is_root and is_left else "      "), True, False)
                else:  # 왼쪽 자식이 없으면 None을 출력
                    print(prefix + (" │    " if not is_root and is_left else "      ") + " ├── None")
                    
                if node.right:  # 오른쪽 자식이 있는 경우
                    self.print_binary_tree_structure(node.right, prefix + (" │    " if not is_root and is_left else "      "), False, False)
                else:  # 오른쪽 자식이 없으면 None을 출력
                    print(prefix + (" │    " if is_left else "      ") + " └── None")

                    
    def printInfo(self, index="Take me somewhere nice"):
        print("\n"+str(index))
        self.print_binary_tree_structure(self.root)
        # tree_structure = str(self).split("\n")  # __str__에서 나온 결과를 줄 단위로 나눔
        # for line in tree_structure:
        #     print("\n    - Tree Structure  | " + line, end="")  # 각 줄에 들여쓰기 추가
        print("=>  info")
        print("    - Preorder        | ", end="")
        self.preorder(self.root)
        print("\n    - inorder         | ", end="")
        self.inorder(self.root)
        print("\n    - Tree height     | ", self.height)
        print("    - Number of Node  | ", self.size, "\n")




    def print_binary_tree_structure(self, node, prefix="", is_left=True, is_root=True):
        if node is not None:
            # 루트 노드 출력 (접두어 없이)
            # 루트 노드는 트리의 최상단에 있으므로 그 앞에 아무런 접두어 없이 출력해야 한다.
            # is_root가 True인 경우, 접두어 없이 루트 노드를 출력.
            if is_root:
                print(f"└── {node}(h={node.height})")
            else:
                # 자식 노드들 출력
                # 루트 노드가 아닌 경우, 접두어를 추가하여 출력.
                # 만약 is_left가 True이면, 왼쪽 자식이므로 "├── "을 접두어로 추가하여 출력.
                # is_left가 False이면, 오른쪽 자식이므로 "└── "을 접두어로 추가하여 출력.
                print(prefix + ("├── " if is_left else "└── ") + f"{node}(h={node.height})")

            # 자식 노드가 있을 경우에만 재귀적으로 자식 노드를 출력
            # node.left 또는 node.right가 존재하는 경우에만 자식 노드들을 출력하기 위해 조건을 체크.
            if node.left or node.right:
                # 왼쪽 자식이 있는 경우
                if node.left:
                    # 왼쪽 자식 노드에 대해서는 다시 동일한 함수를 호출하여 출력.
                    # prefix에 "│    " 또는 "     "을 추가하여 출력. 이는 형제가 있는 경우 트리 구조를 올바르게 표현하기 위함.
                    # 루트 노드가 아닌 경우, 왼쪽 자식이므로 "│    "를 추가하여 자식이 있다는 것을 시각적으로 표시.
                    # 루트 노드의 경우는 prefix에 "     " (공백)만 추가하여 출력, 루트는 자식 노드와 다르게 처리해야 트리 구조가 잘 보임.
                    self.print_binary_tree_structure(node.left, prefix + ("│    " if not is_root else "     "), True, False)
                else:
                    # 왼쪽 자식이 없을 경우 "None"을 출력하여 빈 자리를 시각적으로 표현.
                    # 이는 해당 노드의 왼쪽에 자식이 없는 경우를 보여주기 위함.
                    print(prefix + ("│    " if not is_root else "     ") + "├── None")
                # 오른쪽 자식이 있는 경우
            if node.right:
                # 오른쪽 자식 노드는 더 이상 아래로 이어지는 선이 없도록 접두어 처리
                self.print_binary_tree_structure(node.right, prefix + ("    " if not is_root and is_left else "     "), False, False)
            else:
                # 오른쪽 자식이 없을 때 'None'을 출력
                print(prefix + ("    " if not is_root and is_left else "     ") + "└── None")
                
    def print_binary_tree_structure(self, node, prefix="", is_left=True):
        if node is not None:
            # 현재 노드 출력
            if self.root == node:
                print(prefix + "└── " + f"{node}(h={node.height})")
            else:
                print(prefix + ("├── " if is_left else "└── ") + f"{node}(h={node.height})")
            
            
            if node.left or node.right :
                if node.left:  # 왼쪽 자식이 있는 경우
                    self.print_binary_tree_structure(node.left, prefix + ("│    " if is_left else "     "), True)
                else:  # 왼쪽 자식이 없으면 None을 출력
                    print(prefix + ("│    " if is_left else "    ") + "└── None")
                    
                if node.right:  # 오른쪽 자식이 있는 경우
                    self.print_binary_tree_structure(node.right, prefix + ("│    " if is_left else "     "), False)
                else:  # 오른쪽 자식이 없으면 None을 출력
                    print(prefix + ("│    " if is_left else "     ") + "└── None")


In [119]:
B = BST()
print(B)
B.insert(50)
B.insert(25)
B.insert(35)
B.insert(30)
x = B.insert(75)
B.insert(10)
y = B.insert(11)
B.insert(9)
B.insert(10)
B.printInfo("BST by insert()")


empty

BST by insert()
──── 50(h=3)
       ├── 25(h=2)
       │     ├── 10(h=1)
       │     │     ├── 9(h=0)
       │     │     └── 11(h=0)
       │     └── 35(h=1)
       │           ├── 30(h=0)
       │           └── None
       └── 75(h=0)
=>  info
    - Preorder        | 50 25 10 9 11 35 30 75 
    - inorder         | 9 10 11 25 30 35 50 75 
    - Tree height     |  3
    - Number of Node  |  8 



In [120]:
B.printInfo("Before Delting")
h = B.deleteByMerging(25)
B.printInfo("Delte by using Key 25")
h = B.deleteByMerging(30)
B.printInfo("Delte by using Key 30")
h = B.deleteByMerging(50)
h = B.deleteByMerging(35)
h = B.deleteByMerging(10)
h = B.deleteByMerging(9)
h = B.deleteByMerging(11)
B.printInfo("Delte by using Key")




Before Delting
──── 50(h=3)
       ├── 25(h=2)
       │     ├── 10(h=1)
       │     │     ├── 9(h=0)
       │     │     └── 11(h=0)
       │     └── 35(h=1)
       │           ├── 30(h=0)
       │           └── None
       └── 75(h=0)
=>  info
    - Preorder        | 50 25 10 9 11 35 30 75 
    - inorder         | 9 10 11 25 30 35 50 75 
    - Tree height     |  3
    - Number of Node  |  8 

Unbalanced Point :  11

Delte by using Key 25
──── 50(h=4)
       ├── 10(h=3)
       │     ├── 9(h=0)
       │     └── 11(h=2)
       │           ├── None
       │           └── 35(h=1)
       │                 ├── 30(h=0)
       │                 └── None
       └── 75(h=0)
=>  info
    - Preorder        | 50 10 9 11 35 30 75 
    - inorder         | 9 10 11 30 35 50 75 
    - Tree height     |  4
    - Number of Node  |  7 

Unbalanced Point :  None

Delte by using Key 30
──── 50(h=4)
       ├── 10(h=3)
       │     ├── 9(h=0)
       │     └── 11(h=2)
       │           ├── None
       │      